# Meme Generator Lab - Instructor Notebook
Ref repository: https://github.com/IFML-UT/MLLAcademy-2025

This notebook is used to simulate student interactions with the meme generator pipeline.
Use it to validate that the full captioning stack is working before students begin.

**How It Works**:
- Inputs a freeform meme idea or phrase
- Classifies it into a pre-approved topic
- Uses LLaMA 3.1 8B Instruct (via Hugging Face) to generate 3 clean meme captions
- Filters for profanity or off-topic content

In [1]:
# Setup
# 🛠️ Install dependencies (for Colab or Drive-mount workflows)
!pip install -q \
  torch \
  transformers==4.50.0 \
  huggingface_hub==0.20.3 \
  sentence-transformers==2.2.2 \
  accelerate==0.21.0 \
  better_profanity \
  ipywidgets==7.6.5

import os
import sys
import re
import json
from pathlib import Path

# Ensure utils folder is in path for import
sys.path.append(str(Path("..", "utils").resolve()))
# Import python file for generation and safe captioning
from safe_caption_generator import safe_caption_generator

ERROR: Cannot install huggingface_hub==0.20.3 and transformers==4.50.0 because these package versions have conflicting dependencies.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
Loading embedding model for semantic topic matching...


In [6]:
# Helper - for printing captions cleanly and export the results to a JSON file
# This file will be used later when we generate the images 

def print_captions(captions):
    with open("../captions.json", "w") as f:
        json.dump(captions, f)
    print("\n---\n\n")
    for i, c in enumerate(captions, 1):
        print(f"Caption {i}: {c}\n")

In [ ]:
# --- Now we are going to run the safe caption generator based on your input ---

# 1. Type in a prompt between the quotes assiging it to the variable user_input
# This will then run the caption generator and print the captions
# Note: This may take anywhere from 30 seconds to 2 minutes depending on your prompt and notebook compute resources
try: 
    user_input = "summer school or summer break?"
    print(f"Testing prompt: '{user_input}'")
    captions = safe_caption_generator(user_input, num_captions=3)
    print_captions(captions)

except ValueError as e:
    print(f"⚠️ Error: {e}")

Testing prompt: 'summer school or summer break?'

---


Caption 1: Summer school: where you pay to relearn what you forgot, and also get sunburned, but still no homework

Caption 2: Better a tan, than a GPA

Caption 3: Is summer school like when you burn toast but you're too lazy to make another piece. It's like, why fix it if you can just make do with it?



In [ ]:
# Interactive Prompt (for Demo in class)
from IPython.display import display
import ipywidgets as widgets

input_box = widgets.Text(value='', placeholder='Enter your meme idea...', description='Prompt:')
run_button = widgets.Button(description="Generate")
output = widgets.Output()

def run_on_click(b):
    output.clear_output()
    with output:
        try:
            captions = safe_caption_generator(input_box.value)
            # for idx, c in enumerate(captions, 1): # backup code to print each caption rather than use function
            #   print(f"{idx}. {c}")
            print_captions(captions)
        except Exception as e:
            print(f"⚠️ Error: {e}")

run_button.on_click(run_on_click)
display(input_box, run_button, output)

Text(value='', description='Prompt:', placeholder='Enter your meme idea...')

Button(description='Generate', style=ButtonStyle())

Output()

## Troubleshooting Guide

- If you get a profanity or topic error, verify the input is:
  - Clean (no banned phrases)
  - Topically close to: studying, group projects, sports, coding, school, etc.

- If you get an API error:
  - Ensure `hf_token.txt` exists and contains a valid Hugging Face token; if the token is missing, please ask for a new token.
  - Ensure `.gitignore` excludes it from version control

- If you get no captions back:
  - Check output formatting with `print(repr(captions))`
  - Rerun cell — model output may vary by seed

---
✅ Instructor notebook complete. Move on to Notebook A when you're ready.
